In [165]:
import os
import pandas as pd

In [166]:
vibration_data = []
machine_status = []

data_list = os.listdir("./data/xep-vibration-data")

for w in data_list:
    if "week" in w:
        week_data_list = os.listdir(f"./data/xep-vibration-data/{w}")
        for d in week_data_list:
            if "vibrationdata" in d:
                vibration_data.append(f"./data/xep-vibration-data/{w}/{d}")
            elif "machinestatus" in d:
                machine_status.append(f"./data/xep-vibration-data/{w}/{d}")

In [167]:
vibration_df = pd.concat([pd.read_csv(v) for v in vibration_data], ignore_index=True)
machine_status_df = pd.concat([pd.read_csv(m) for m in machine_status], ignore_index=True)

In [168]:
vibration_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107316 entries, 0 to 107315
Data columns (total 41 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Timestamp                107275 non-null  float64
 1   SensorID                 107304 non-null  object 
 2   SequenceNumber           107316 non-null  object 
 3   TotalLength              107275 non-null  float64
 4   SourceAddress            0 non-null       float64
 5   TempHumiRange            107275 non-null  float64
 6   TempHumiStatus           107275 non-null  float64
 7   TempHumiEvent            107275 non-null  float64
 8   TempHumiSenVal           107275 non-null  float64
 9   XSenEvent                107275 non-null  float64
 10  XOAVelocity              107275 non-null  float64
 11  XPeakmg                  107275 non-null  float64
 12  XRMSmg                   107275 non-null  float64
 13  XKurtosis                107275 non-null  float64
 14  XCre

In [169]:
machine_status_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332190 entries, 0 to 1332189
Data columns (total 23 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   Timestamp  1332190 non-null  int64  
 1   UID        1332190 non-null  object 
 2   machine1   451137 non-null   float64
 3   machine2   451137 non-null   float64
 4   machine3   451137 non-null   float64
 5   machine4   451137 non-null   float64
 6   machine5   451137 non-null   float64
 7   machine6   452560 non-null   float64
 8   machine7   452560 non-null   float64
 9   machine8   452560 non-null   float64
 10  machine9   452560 non-null   float64
 11  machine10  452560 non-null   float64
 12  machine11  452560 non-null   float64
 13  machine12  452560 non-null   float64
 14  machine13  452560 non-null   float64
 15  machine14  428493 non-null   float64
 16  machine15  428493 non-null   float64
 17  machine16  428493 non-null   float64
 18  machine17  428493 non-null   float64
 19  

In [170]:
vibration_df = vibration_df.dropna(subset=["Timestamp"])
machine_status_df = machine_status_df.dropna(subset=["Timestamp"])

In [171]:
vibration_df.duplicated(["Timestamp", "SensorID"]).value_counts()

False    107254
True         21
dtype: int64

In [172]:
first = vibration_df.loc[
    vibration_df.duplicated(["Timestamp", "SensorID"], keep="first")
].reset_index(drop=True)

last = vibration_df.loc[
    vibration_df.duplicated(["Timestamp", "SensorID"], keep="last")
].reset_index(drop=True)

first.equals(last)

False

In [173]:
vibration_df.loc[
    vibration_df.duplicated(["Timestamp", "SensorID"], keep="first") |
    vibration_df.duplicated(["Timestamp", "SensorID"], keep="last")
].to_csv("test.csv", index=False)

In [174]:
first = vibration_df.loc[
    vibration_df.duplicated(["Timestamp", "SensorID"], keep="first") &
    vibration_df["Timestamp"] != 1676435835000
].reset_index(drop=True)

last = vibration_df.loc[
    vibration_df.duplicated(["Timestamp", "SensorID"], keep="last") &
    vibration_df["Timestamp"] != 1676435835000
].reset_index(drop=True)

first.equals(last)

True

In [175]:
vibration_df = vibration_df.drop_duplicates(["Timestamp", "SensorID"])

In [176]:
machine_status_df.duplicated(["Timestamp", "UID"]).value_counts()

False    1332178
True          12
dtype: int64

In [177]:
first = machine_status_df.loc[
    machine_status_df.duplicated(["Timestamp", "UID"], keep="first")
].reset_index(drop=True)

last = machine_status_df.loc[
    machine_status_df.duplicated(["Timestamp", "UID"], keep="last")
].reset_index(drop=True)

first.equals(last)

True

In [178]:
machine_status_df = machine_status_df.drop_duplicates(["Timestamp", "UID"])

In [179]:
sensor1 = machine_status_df.loc[machine_status_df["UID"] == "sensor1"]
sensor2 = machine_status_df.loc[machine_status_df["UID"] == "sensor2"]
sensor3 = machine_status_df.loc[machine_status_df["UID"] == "sensor3"]

print(f"sensor1.shape: {sensor1.shape}")
print(f"sensor2.shape: {sensor2.shape}")
print(f"sensor3.shape: {sensor3.shape}")

sensor1.shape: (451133, 23)
sensor2.shape: (452556, 23)
sensor3.shape: (428489, 23)


In [180]:
len(set(sensor1["Timestamp"]).intersection(set(sensor2["Timestamp"])))

1957

In [181]:
len(set(sensor1["Timestamp"]).intersection(set(sensor3["Timestamp"])))

1040

In [182]:
vibration_df["Timestamp"] = vibration_df["Timestamp"] / 1000
vibration_df["Timestamp"] = vibration_df["Timestamp"].astype("int64")
vibration_df["DateTime"] = pd.to_datetime(vibration_df["Timestamp"], unit="s")
vibration_df = vibration_df.sort_values("DateTime", ignore_index=True)

machine_status_df["Timestamp"] = machine_status_df["Timestamp"] / 1000
machine_status_df["Timestamp"] = machine_status_df["Timestamp"].astype("int64")
machine_status_df["DateTime"] = pd.to_datetime(machine_status_df["Timestamp"], unit="s")
machine_status_df = machine_status_df.sort_values("DateTime", ignore_index=True)

In [183]:
sensor1 = machine_status_df.loc[machine_status_df["UID"] == "sensor1"]
sensor2 = machine_status_df.loc[machine_status_df["UID"] == "sensor2"]
sensor3 = machine_status_df.loc[machine_status_df["UID"] == "sensor3"]

print(f"sensor1.shape: {sensor1.shape}")
print(f"sensor2.shape: {sensor2.shape}")
print(f"sensor3.shape: {sensor3.shape}")

sensor1.shape: (451133, 24)
sensor2.shape: (452556, 24)
sensor3.shape: (428489, 24)


In [184]:
len(set(sensor1["Timestamp"]).intersection(set(sensor2["Timestamp"])))

200240

In [185]:
len(set(sensor1["Timestamp"]).intersection(set(sensor3["Timestamp"])))

114301

In [186]:
front_cols = ["Timestamp", "DateTime"]
vibration_df = vibration_df[front_cols + [col for col in vibration_df if col not in front_cols]]
machine_status_df = machine_status_df[front_cols + [col for col in machine_status_df if col not in front_cols]]

In [187]:
vibration_df

,Timestamp,DateTime,SensorID,SequenceNumber,TotalLength,SourceAddress,TempHumiRange,TempHumiStatus,TempHumiEvent,TempHumiSenVal,...,ZKurtosis,ZCrestFactor,ZSkewness,ZDeviation,ZPeaktoPeakDisplacement,LogIndex,DeviceEvents,DevicePowerSrc,DeviceBatteryVolt,DeviceTime
0,1668729853,2022-11-18 00:04:13,sensor-2,135,88.0,NaN,0.0,0.0,0.0,34562.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3544.0,1.668730e+09
1,1668729866,2022-11-18 00:04:26,sensor-4,255,88.0,NaN,0.0,0.0,0.0,29875.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3558.0,1.668730e+09
2,1668730437,2022-11-18 00:13:57,sensor-1,72,88.0,NaN,0.0,0.0,0.0,30062.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3503.0,1.668730e+09
3,1668730753,2022-11-18 00:19:13,sensor-2,136,88.0,NaN,0.0,0.0,0.0,34312.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3544.0,1.668731e+09
4,1668730766,2022-11-18 00:19:26,sensor-4,0,88.0,NaN,0.0,0.0,0.0,29687.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3558.0,1.668731e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107249,1698622637,2023-10-29 23:37:17,sensor-4,0,88.0,NaN,0.0,0.0,0.0,28437.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3493.0,1.564529e+09
107250,1698622651,2023-10-29 23:37:31,sensor-1,2,88.0,NaN,0.0,0.0,0.0,28500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3505.0,1.564529e+09
107251,1698623438,2023-10-29 23:50:38,sensor-2,204,88.0,NaN,0.0,0.0,0.0,28937.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3463.0,1.566833e+09
107252,1698623538,2023-10-29 23:52:18,sensor-4,1,88.0,NaN,0.0,0.0,0.0,28437.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3493.0,1.564530e+09


In [188]:
machine_status_df

,Timestamp,DateTime,UID,machine1,machine2,machine3,machine4,machine5,machine6,machine7,...,machine12,machine13,machine14,machine15,machine16,machine17,machine18,machine19,machine20,machine21
0,1668729636,2022-11-18 00:00:36,sensor3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1668729698,2022-11-18 00:01:38,sensor3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1668729760,2022-11-18 00:02:40,sensor3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1668729822,2022-11-18 00:03:42,sensor3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1668729884,2022-11-18 00:04:44,sensor3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332173,1696204810,2023-10-02 00:00:10,sensor3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
1332174,1696204810,2023-10-02 00:00:10,sensor1,1.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1332175,1696204870,2023-10-02 00:01:10,sensor2,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1332176,1696204870,2023-10-02 00:01:10,sensor3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


In [189]:
vibration_df["converted"] = vibration_df["DateTime"].dt.strftime("%Y-%m-%d %H:%M")
machine_status_df["converted"] = machine_status_df["DateTime"].dt.strftime("%Y-%m-%d %H:%M")

In [190]:
sensor1 = machine_status_df.loc[machine_status_df["UID"] == "sensor1"]
sensor2 = machine_status_df.loc[machine_status_df["UID"] == "sensor2"]
sensor3 = machine_status_df.loc[machine_status_df["UID"] == "sensor3"]

print(f"sensor1.shape: {sensor1.shape}")
print(f"sensor2.shape: {sensor2.shape}")
print(f"sensor3.shape: {sensor3.shape}")

sensor1.shape: (451133, 25)
sensor2.shape: (452556, 25)
sensor3.shape: (428489, 25)


In [191]:
len(set(sensor1["converted"]).intersection(set(sensor2["converted"])))

420697

In [192]:
len(set(sensor1["converted"]).intersection(set(sensor3["converted"])))

425679

In [193]:
vibration_df.duplicated(["converted", "SensorID"]).value_counts()

False    107242
True         12
dtype: int64

In [194]:
vibration_df.loc[
    vibration_df.duplicated(["converted", "SensorID"], keep="first") |
    vibration_df.duplicated(["converted", "SensorID"], keep="last")
].to_csv("test.csv", index=False)

In [195]:
machine_status_df.duplicated(["converted", "UID"]).value_counts()

False    1274684
True       57494
dtype: int64

In [196]:
mapped_df = vibration_df.merge(machine_status_df, on=["converted"], how="inner", suffixes=["_v", "_s"])
mapped_df = mapped_df.sort_values(["Timestamp_v", "Timestamp_s"], ignore_index=True)

front_cols = ["converted", "Timestamp_v", "Timestamp_s", "DateTime_v", "DateTime_s"]
mapped_df = mapped_df[front_cols + [col for col in mapped_df if col not in front_cols]]
mapped_df

,converted,Timestamp_v,Timestamp_s,DateTime_v,DateTime_s,SensorID,SequenceNumber,TotalLength,SourceAddress,TempHumiRange,...,machine12,machine13,machine14,machine15,machine16,machine17,machine18,machine19,machine20,machine21
0,2022-11-18 00:04,1668729853,1668729884,2022-11-18 00:04:13,2022-11-18 00:04:44,sensor-2,135,88.0,NaN,0.0,...,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-11-18 00:04,1668729866,1668729884,2022-11-18 00:04:26,2022-11-18 00:04:44,sensor-4,255,88.0,NaN,0.0,...,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-11-18 00:13,1668730437,1668730380,2022-11-18 00:13:57,2022-11-18 00:13:00,sensor-1,72,88.0,NaN,0.0,...,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-11-18 00:19,1668730753,1668730753,2022-11-18 00:19:13,2022-11-18 00:19:13,sensor-2,136,88.0,NaN,0.0,...,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-11-18 00:19,1668730766,1668730753,2022-11-18 00:19:26,2022-11-18 00:19:13,sensor-4,0,88.0,NaN,0.0,...,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292825,2023-10-01 23:54,1696204477,1696204450,2023-10-01 23:54:37,2023-10-01 23:54:10,sensor-4,176,88.0,NaN,0.0,...,NaN,NaN,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
292826,2023-10-01 23:54,1696204477,1696204450,2023-10-01 23:54:37,2023-10-01 23:54:10,sensor-4,176,88.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292827,2023-10-01 23:54,1696204497,1696204450,2023-10-01 23:54:57,2023-10-01 23:54:10,sensor-1,176,88.0,NaN,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292828,2023-10-01 23:54,1696204497,1696204450,2023-10-01 23:54:57,2023-10-01 23:54:10,sensor-1,176,88.0,NaN,0.0,...,NaN,NaN,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0


In [197]:
len(mapped_df["Timestamp_v"].unique())

93156

In [198]:
mapped_df[mapped_df["Timestamp_v"]==1669578553]

,converted,Timestamp_v,Timestamp_s,DateTime_v,DateTime_s,SensorID,SequenceNumber,TotalLength,SourceAddress,TempHumiRange,...,machine12,machine13,machine14,machine15,machine16,machine17,machine18,machine19,machine20,machine21
21758,2022-11-27 19:49,1669578553,1669578542,2022-11-27 19:49:13,2022-11-27 19:49:02,sensor-2,54,88.0,NaN,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21759,2022-11-27 19:49,1669578553,1669578546,2022-11-27 19:49:13,2022-11-27 19:49:06,sensor-2,54,88.0,NaN,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21760,2022-11-27 19:49,1669578553,1669578555,2022-11-27 19:49:13,2022-11-27 19:49:15,sensor-2,54,88.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21761,2022-11-27 19:49,1669578553,1669578558,2022-11-27 19:49:13,2022-11-27 19:49:18,sensor-2,54,88.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21762,2022-11-27 19:49,1669578553,1669578562,2022-11-27 19:49:13,2022-11-27 19:49:22,sensor-2,54,88.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21763,2022-11-27 19:49,1669578553,1669578577,2022-11-27 19:49:13,2022-11-27 19:49:37,sensor-2,54,88.0,NaN,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [199]:
vibration_df.to_csv("./data/vibrationdata.csv", index=False)
machine_status_df.to_csv("./data/machinestatus.csv", index=False)
mapped_df.to_csv("./data/mapped.csv", index=False)